In [13]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline


In [14]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import torch 
import torch.nn.functional as F
from torch import nn
import pandas as pd
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
from iou_calculator.utils import Merge, show_im, show_im_origin, check_box, clip_corner



In [15]:
cd ..

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline


In [16]:
cd utils

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/utils


In [17]:
from model_func import *
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw
import plotly.express as px


In [18]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline


In [19]:
class NeuralNetwork_BrightnessContrast(nn.Module):
    '''
    New model that takes as input a brightness or constrat value and apply it to a specific image
    using the linear_perturbation layer
    (aka the weight and biases of the linear_perturbation layer are set to specific values to encode
    brightness or contrast for a specific image)
    '''
    def __init__(self, classif=True):
        super(NeuralNetwork_BrightnessContrast, self).__init__()
        self.classif=classif
        seed = 0
        torch.manual_seed(seed)
        padding = 1
        self.linear_perturbation = nn.Linear(1,90*90)
        self.conv0 = nn.Conv2d(1, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool0 = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(16, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn.Linear(7744,256)
        if self.classif:
            self.linear = nn.Linear(256, 10)
        else:
            self.linear_all = nn.Linear(256, 4) 

    
    def forward(self, alpha):
        layer = self.linear_perturbation(alpha)
        layer = layer.view((-1, 1, 90, 90))
        layer = self.conv0(layer)
        layer = F.relu(self.pool0(layer))
        layer = self.conv1(layer)
        layer = F.relu(self.pool1(layer))
        layer = self.flatten(layer)
        layer = self.linear_relu_stack(layer)
        layer = F.relu(layer)
        if self.classif:
            logits = self.linear(layer)
        else:
            logits = self.linear_all(layer)
        return logits

In [20]:
model_test = NeuralNetwork_BrightnessContrast(classif=False)
model_test(torch.Tensor([0.001]))

tensor([[-0.1079, -0.0068, -0.0729, -0.0156]], grad_fn=<AddmmBackward0>)

## Digit model

In [21]:
model_digit = NeuralNetwork_BrightnessContrast(classif=True)

filename = 'toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
digit_config = model_torch_load.state_dict()
digit_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
digit_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_digit.load_state_dict(digit_config)
#model_torch_load.state_dict()

<All keys matched successfully>

## Corner Model

In [22]:
model_corners = NeuralNetwork_BrightnessContrast(classif=False)

filename = 'toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
corner_config = model_torch_load.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
corner_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
corner_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners.load_state_dict(corner_config)
#model_torch_load.state_dict()


<All keys matched successfully>

## Load Data

In [23]:
cd utils

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/utils


In [24]:
train_df = pd.read_csv("train.csv")
dict_df_10 = {}
for label in np.unique(list(train_df["label"])): 
    dict_df_10[label] = train_df[train_df["label"] == label].head(10)

In [25]:
train_df_100 = pd.concat(dict_df_10.values())
trainingData = CustomMnistDataset_OL(train_df_100)
train_dataloader = DataLoader(trainingData, batch_size=1, shuffle=False)
iterator = iter(train_dataloader)
print(len(train_dataloader))

100


In [26]:
data = next(iter(train_dataloader))
X_0, _ = data
data = next(iter(train_dataloader))
X_1, Y_1 = data

## Brightness

brightness(x; alpha)= x+ alpha

thus for linear_perturbation

linear_perturbation.weight = [1..1]

linear_perturbation.weight = x

In [27]:
def set_brightness(model, image):
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any brightness value
    image_flatten = image.view((90*90,))
    brightness_config = model.state_dict()
    brightness_config['linear_perturbation.bias']=image_flatten
    brightness_config['linear_perturbation.weight']=torch.Tensor(np.ones((90*90,1), dtype='float32'))
    model.load_state_dict(brightness_config)
    

## AutoLirpa on brightness

In [65]:
variations_brightness = np.linspace(0, 0.007,101)

In [62]:
def expe_brightness(df=train_df_100[:3]): 

    tensor_init_brightness = torch.tensor([[0.0]]).float().to('cpu')
    
    dico_brightness = {"image":[], "gt_box":[], "image_id":[], "alpha_range":[], "brightness_variation":[], "lb":[], "ub":[]}

    for image_id in range(len(df)):
            model_lirpa_corners = BoundedModule(model_corners, tensor_init_brightness)
            data = next(iterator)
            X, y = data
            mask = np.sign(X[0]) # new

            set_brightness(model_corners, X)
            gt_logit, gt_box = y
            gt_box = gt_box.detach().numpy()[0]
        
            for brightness_variations in variations_brightness:
                print(f"*******{brightness_variations} OK******")
                ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=brightness_variations) #attention espilon c'est X_brightness
                input_lirpa_brightness = BoundedTensor(torch.tensor([0.0]), ptb_brightness)
                
                lb_brightness, ub_brightness = model_lirpa_corners.compute_bounds(x=(input_lirpa_brightness,),
                                                                    IBP=True, method='crown')
                dico_brightness["image"].append(X)
                dico_brightness["gt_box"].append(gt_box)
                dico_brightness["image_id"].append(image_id)
                dico_brightness["alpha_range"].append(str([tensor_init_brightness.item()-brightness_variations,
                                                           tensor_init_brightness.item()+brightness_variations])) 
                dico_brightness["brightness_variation"].append(brightness_variations)
                dico_brightness["lb"].append(lb_brightness[0].tolist())
                dico_brightness["ub"].append(ub_brightness[0].tolist())
                
                try:
                    show_im_origin(X, f"/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline/brightness_manip_todelete/images_try/{image_id}/{brightness_variations}.png", 
                                   gt_box, 
                                   lb_brightness[0].tolist(), 
                                   ub_brightness[0].tolist())
                except ValueError as e:
                    print(f"--------------{image_id}/{brightness_variations}")


    df = pd.DataFrame(dico_brightness)
    return(df)

df_brightness = expe_brightness()

*******0.0 OK******
be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.0)>
BoundInput(name="/0")
be careful added by Noemie
be careful added by Noemie
*******7.000000000000001e-05 OK******
be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0

In [64]:
df_brightness

,image,gt_box,image_id,alpha_range,brightness_variation,lb,ub
0,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[29.0, 5.0, 57.0, 33.0]",0,"[0.0, 0.0]",0.00000,"[31.056400299072266, 5.088156700134277, 58.751...","[31.056400299072266, 5.088156700134277, 58.751..."
1,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[29.0, 5.0, 57.0, 33.0]",0,"[-7.000000000000001e-05, 7.000000000000001e-05]",0.00007,"[30.870079040527344, 5.003620624542236, 58.494...","[31.65795135498047, 5.2550272941589355, 59.742..."
2,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[29.0, 5.0, 57.0, 33.0]",0,"[-0.00014000000000000001, 0.00014000000000000001]",0.00014,"[30.630109786987305, 4.908689975738525, 58.197...","[32.33844757080078, 5.490225791931152, 60.9305..."
3,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[29.0, 5.0, 57.0, 33.0]",0,"[-0.00021, 0.00021]",0.00021,"[30.379674911499023, 4.795416355133057, 57.902...","[33.11665725708008, 5.77054500579834, 62.32976..."
4,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[29.0, 5.0, 57.0, 33.0]",0,"[-0.00028000000000000003, 0.00028000000000000003]",0.00028,"[30.103736877441406, 4.667764663696289, 57.579...","[34.015838623046875, 6.119705677032471, 63.955..."
...,...,...,...,...,...,...,...
298,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[37.0, 41.0, 65.0, 69.0]",2,"[-0.00672, 0.00672]",0.00672,"[-48.346012115478516, -36.773826599121094, -25...","[210.69570922851562, 206.68641662597656, 340.9..."
299,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[37.0, 41.0, 65.0, 69.0]",2,"[-0.006790000000000001, 0.006790000000000001]",0.00679,"[-49.478912353515625, -37.7959098815918, -26.8...","[212.7745361328125, 208.664794921875, 344.2017..."
300,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[37.0, 41.0, 65.0, 69.0]",2,"[-0.006860000000000001, 0.006860000000000001]",0.00686,"[-50.637901306152344, -38.84535598754883, -28....","[214.81712341308594, 210.61341857910156, 347.3..."
301,"[[[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.,...","[37.0, 41.0, 65.0, 69.0]",2,"[-0.00693, 0.00693]",0.00693,"[-51.814178466796875, -39.91312026977539, -29....","[216.85989379882812, 212.56224060058594, 350.5..."


In [67]:
image = df_brightness["image"][0]

In [105]:
def bound_whitenoise(model_box, model_digit, X, eps):
    X_lirpa = X.float().to('cpu')

    mask = np.sign(X_lirpa[0])
    lb_ = 0.*X_lirpa
    ub_ = mask

    model_lirpa_corner = BoundedModule(model_box, X_lirpa)

    x_min = X_lirpa - eps*mask
    x_max = X_lirpa + eps*mask

    ptb = PerturbationLpNorm(norm=np.inf, eps=eps, x_L=x_min, x_U=x_max)
    input_lirpa = BoundedTensor(X_lirpa, ptb)

    lb_box, ub_box = model_lirpa_corner.compute_bounds(x=(input_lirpa,),IBP=True, method='crown')



    return lb_box.detach().numpy()[0], ub_box.detach().numpy()[0]


In [123]:
def bound_whitenoise_withoutwLxU(model_box, model_digit, X, eps):
    X_lirpa = X.float().to('cpu')

    mask = np.sign(X_lirpa[0])
    lb_ = 0.*X_lirpa
    ub_ = mask

    model_lirpa_corner = BoundedModule(model_box, X_lirpa)

    x_min = X_lirpa - eps*mask
    x_max = X_lirpa + eps*mask

    ptb = PerturbationLpNorm(norm=np.inf, eps=eps)
    input_lirpa = BoundedTensor(X_lirpa, ptb)

    lb_box, ub_box = model_lirpa_corner.compute_bounds(x=(input_lirpa,),IBP=True, method='crown')



    return lb_box.detach().numpy()[0], ub_box.detach().numpy()[0]

In [106]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline


In [107]:
from detection.NeuralNetwork_OL_v2 import NeuralNetwork_OL_v2
filename = 'detection/weights/toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
model_digit = NeuralNetwork_OL_v2(classif=True)
model_digit.load_state_dict(model_torch_load.state_dict())

filename = 'detection/weights/toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
model_box = NeuralNetwork_OL_v2(classif=False)
model_box.load_state_dict(model_torch_load.state_dict())

<All keys matched successfully>

In [122]:
loci = 55
variation = df_brightness["brightness_variation"][loci], loci
variations

(0.0038500000000000006, 55)

In [ ]:
lb_without, ub_without = bound_whitenoise(model_box, model_digit, image, 0)

In [ ]:
lb_with, ub_with = bound_whitenoise(model_box, model_digit, image, variation[0])

In [124]:
lb_with_withoutwLxU, ub_with_withoutwLxU = bound_whitenoise_withoutwLxU(model_box, model_digit, image, variation[0])

be careful Noémie is doing shit on auto lirpa [BoundInput(name="/input.1"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8")] ['/input.1', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8']
<BoundedTensor: BoundedTensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
                 [0., 0., 0.,  ..., 0., 0., 0.],
                 [0., 0., 0.,  ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0.,  ..., 0., 0., 0.],
                 [0., 0., 0.,  ..., 0., 0., 0.],
                 [0., 0., 0.,  ..., 0., 0., 0.]]]]), PerturbationLpNorm(norm=inf, eps=0.0038500000000000006)>
BoundInput(name="/input.1")
be careful added by Noemie
be careful added by Noemie


In [119]:
print("Perturbation WHITE NOISE")
print("Without perturbation: lower bound :", lb_without)
print("Without perturbation:  upper bound :", ub_without)
print(variation[0], ": lower bound :", lb_with )
print(variations[0], ":  upper bound :", ub_with )

Perturbation WHITE NOISE
Without perturbation: lower bound : [31.0564     5.0881567 58.751377  33.04274  ]
Without perturbation:  upper bound : [31.0564     5.0881567 58.751377  33.04274  ]
0.0038500000000000006 : lower bound : [30.858234  4.973539 58.51881  32.8886  ]
(0.0038500000000000006, 55) :  upper bound : [31.36298    5.3296566 59.35131   33.545387 ]


In [127]:
print("Perturbation WHITE NOISE SANS BORNER LE BOUNDED TENSOR")
print("Without perturbation: lower bound :", lb_without)
print("Without perturbation:  upper bound :", ub_without)
print(variation[0], ": lower bound :", lb_with_withoutwLxU)
print(variations[0], ":  upper bound :", ub_with_withoutwLxU )

Perturbation WHITE NOISE SANS BORNER LE BOUNDED TENSOR
Without perturbation: lower bound : [31.0564     5.0881567 58.751377  33.04274  ]
Without perturbation:  upper bound : [31.0564     5.0881567 58.751377  33.04274  ]
0.0038500000000000006 : lower bound : [ -1.8540888 -11.734826   19.288813   11.534316 ]
0.0038500000000000006 :  upper bound : [119.46107  66.2173  202.899   149.75348]


In [120]:
print("Perturbation Brightness")
print("Without perturbation: brightness, lower bound :", df_brightness["lb"][0])
print("Without perturbation: brightness, upper bound :", df_brightness["ub"][0])
print(variations[0], ": brightness, lower bound :", df_brightness["lb"][variations[1]] )
print(variations[0], ": brightness, upper bound :", df_brightness["ub"][variations[1]] )

Perturbation Brightness
Without perturbation: brightness, lower bound : [31.056400299072266, 5.088156700134277, 58.75137710571289, 33.04273986816406]
Without perturbation: brightness, upper bound : [31.056400299072266, 5.088156700134277, 58.75137710571289, 33.04273986816406]
0.0038500000000000006 : brightness, lower bound : [-0.7428011894226074, -11.260683059692383, 21.273143768310547, 12.818248748779297]
0.0038500000000000006 : brightness, upper bound : [119.1829605102539, 65.81848907470703, 202.53155517578125, 149.4157257080078]
